In [3]:
with open('input.txt') as file:
    data = [x.strip('\n') for x in file.readlines()]

In [4]:
data[0]

'[1518-11-22 00:00] Guard #1231 begins shift'

In [5]:
import pandas as pd

In [6]:
timestamp = [x.split(']')[0].strip('[') for x in data]

In [10]:
import re

In [30]:
def find_guard(string):
    match = re.search(string=string, pattern='Guard #\d+')
    if match is not None:
        return match.group(0)
    return None

In [31]:
guard = [find_guard(x.split(']')[1]) for x in data]

In [33]:
def find_status(string):
    s = string.split(']')[1]
    if 'begins shift' in s:
        return 'begins shift'
    if 'falls asleep' in s:
        return 'falls asleep'
    if 'wakes up' in s:
        return 'wakes up'
    return None

In [34]:
status = [find_status(x) for x in data]

In [193]:
df = pd.DataFrame({'time': timestamp, 'guard': guard, 'status': status})
df['useful_time'] = df['time'].apply(lambda x: '2018-' + x[5:])
df['useful_time'] = pd.to_datetime(df['useful_time'])
df.sort_values(by='useful_time', ascending=True, inplace=True)
df['guard'] = df.guard.fillna(method='ffill')

In [194]:
df['time_since_last'] = df['useful_time'].diff().dt.seconds.div(60, fill_value=0)

In [195]:
df['time_asleep'] = df.time_since_last * (df.status == 'wakes up')

In [197]:
most_asleep = df.groupby(['guard']).agg({'time_asleep': sum}).sort_values('time_asleep', ascending=False).head(1)
most_asleep_guard = most_asleep.iloc[0].name
most_asleep

,time_asleep
guard,
Guard #3371,526.0


In [208]:
df['prev_time'] = df.useful_time.shift()
df['prev_status'] = df.status.shift()
df['prev_minute'] = df.prev_time.dt.minute

In [210]:
df.head()

,time,guard,status,useful_time,time_since_last,time_asleep,prev_time,prev_status,prev_minute
1022,1518-02-10 23:47,Guard #1033,begins shift,2018-02-10 23:47:00,0.0,0.0,NaT,NaN,NaN
335,1518-02-11 00:05,Guard #1033,falls asleep,2018-02-11 00:05:00,18.0,0.0,2018-02-10 23:47:00,begins shift,47.0
901,1518-02-11 00:28,Guard #1033,wakes up,2018-02-11 00:28:00,23.0,23.0,2018-02-11 00:05:00,falls asleep,5.0
696,1518-02-11 00:43,Guard #1033,falls asleep,2018-02-11 00:43:00,15.0,0.0,2018-02-11 00:28:00,wakes up,28.0
918,1518-02-11 00:52,Guard #1033,wakes up,2018-02-11 00:52:00,9.0,9.0,2018-02-11 00:43:00,falls asleep,43.0


In [211]:
most_asleep_df = df[(df.status == 'wakes up') & (df.guard == most_asleep.iloc[0,].name)]

In [212]:
most_asleep_minutes = most_asleep_df[['prev_minute', 'time_asleep']].reset_index().to_dict('records')

In [215]:
def find_minutes_asleep(record):
    time_asleep = int(record['time_asleep'])
    minute = int(record['prev_minute'])
    if time_asleep + minute >= 60:
        time_asleep = 60 - minute
    return list(range(minute, minute + time_asleep))

In [219]:
mins_where_asleep = [find_minutes_asleep(r) for r in most_asleep_minutes]

In [220]:
count_mins_where_asleep = {}
for mins in mins_where_asleep:
    for m in mins:
        if count_mins_where_asleep.get(m) is None:
            count_mins_where_asleep[m] = 1
        else:
            count_mins_where_asleep[m] += 1

In [221]:
print("Part One:", max(count_mins_where_asleep.values()) * int(most_asleep.iloc[0].name[-4:]))

Part One: 47194


In [223]:
sum(count_mins_where_asleep.values())

526